# DC GAN

Deconvolution GAN

In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe 
tf.enable_eager_execution()
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense
from tensorflow.keras.layers import BatchNormalization, Reshape, UpSampling2D, MaxPool2D, Flatten
import numpy as np 

#  Define Generator

In [2]:
class Generator(Model):
    def __init__(self):
        super(Generator, self).__init__()        
        self.dense1 = Dense(1024, activation='tanh', input_dim=(100))
        self.dense2 = Dense(128*7*7, activation='tanh')
        self.bn1 = BatchNormalization()
        self.reshape1 = Reshape((7,7,128),input_shape=(128*7*7,))
        self.up1 = UpSampling2D(size=(2,2))
        self.conv1 = Conv2D(64,(5,5), padding='same', activation='tanh')
        self.up2 = UpSampling2D(size=(2,2))
        self.conv2 = Conv2D(1,(5,5), padding='same', activation='tanh')
        
    def __call__(self, x):
        x = self.dense1(x)        
        x = self.dense2(x)                
        x = self.bn1(x)        
        x = self.reshape1(x)        
        x = self.up1(x)        
        x = self.conv1(x)        
        x = self.up2(x)        
        x = self.conv2(x)        
        return x 

In [3]:
test = tf.random.normal((1,100))
print(test.shape)
g = Generator()
result = g(test[0:])
print(result.shape)

(1, 100)
(1, 28, 28, 1)


# Define Discrimator

In [4]:
class Discrimator(Model):
    def __init__(self):
        super(Discrimator,self).__init__()
        self.conv1 = Conv2D(64, (5,5), padding='same', activation='tanh')
        self.maxPool1 = MaxPool2D(pool_size=(2,2))
        self.conv2 = Conv2D(128, (5,5), padding='same', activation='tanh')
        self.maxPool2 = MaxPool2D(pool_size=(2,2))
        self.flatten = Flatten()
        self.dense1 = Dense(1024, activation='tanh')
        self.dense2 = Dense(1, activation='sigmoid')
        
    def __call__(self, x):
        x = self.conv1(x)
        x = self.maxPool1(x)
        x = self.conv2(x)
        x = self.maxPool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x 

In [5]:
test = tf.random.normal((1,28,28,1))
print(test.shape)
d = Discrimator()
result = d(test)
print(result.shape)

(1, 28, 28, 1)
(1, 1)


# Define Z (1024)

In [7]:
BATCH_SIZE=32
LATENT_SIZE=100
z = tf.random.uniform(minval=-1,maxval=1,shape=(BATCH_SIZE, LATENT_SIZE), seed=0)

In [8]:
print(z.shape)

(32, 100)


In [9]:
class GAN():
    def __init__(self, g, d):
        self.G = g
        self.D = d
        self.noise_size = 100 
        
    def set_images(self, img_data):
        self.img_data = img_data
        
    def train(self,batch_size=100):
        # Pick image data randomly. 
        images_train = self.img_data[np.random.randint(0, self.img_data.shape[0], size=batch_size), :, :, :] 
        # Generate images from noise. 
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, self.noise_size]) 
        images_fake = self.G(noise) 
        # Train D. 
        x = np.concatenate((images_train, images_fake)) 
        y = np.ones([2*batch_size, 1]) 
        y[batch_size:, :] = 0 
        self.D.trainable = True 
        y_hat = seld.D(x)
        d_loss = y-y_hat
        # Train G. 
        y = np.ones([batch_size, 1]) 
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, self.noise_size]) 
        self.D.trainable = False 
        a_loss = self.AM.train_on_batch(noise, y) 
        return d_loss, a_loss, images_fake        

In [10]:
(x_train, y_train), (x_test, y_test)  = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0 
x_train = x_train.reshape((x_train.shape[0],) + (28, 28, 1)) 
print(x_train.shape) 
# Init network 
gan = GAN(g, d)
gan.set_images(x_train)
# Some parameters. 
epochs = 30 
sample_size = 10 
batch_size = 100 
train_per_epoch = x_train.shape[0] // batch_size 

for epoch in range(0, epochs): 
    print("Epoch:", epoch + 1) 
    total_d_loss = 0.0 
    total_a_loss = 0.0 
    for batch in range(0, train_per_epoch): 
        d_loss, a_loss, imgs = gan.train(batch_size) 
        total_d_loss += d_loss 
        total_a_loss += a_loss 
        total_d_loss /= train_per_epoch 
        total_a_loss /= train_per_epoch 
        print("D Loss: {}, AM Loss: {}".format(total_d_loss, total_a_loss)) 
        # Show generated images. 
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1)) 
        for i in range(0, sample_size): 
            ax[i].set_axis_off() 
            ax[i].imshow(imgs[i].reshape((28, 28))); 
            plt.show() 
            plt.close(fig)

(60000, 28, 28, 1)
Epoch: 1


TypeError: __call__() takes 2 positional arguments but 3 were given